# CHAPTER 7. Market Basket Analysis

### 연관성분석(Association Analysis)

- 연관성 규칙을 통해 하나의 거래나 사건에 포함되어 있는 둘 이상의 품목 간 상호 연관성을 발견해 내는 것.

###  연관규칙의 현장 적용사례
- 어떤 상품들이 함께 잘 팔리나?
-  제품을 구입한 고객에게 함께 팔 수 있는 것은?
    - ** 상품 진열에 영향 **


- 연관 규칙을 발견해 내는 작업이란, 그 데이터 안에 있는 항목 간의
종속관계를 찾아 내는 작업
- 마케팅에서는 고객의 장바구니 안에 들어있는 품목간의 관계를 알아본다는 의미에서 장바구니 분석이라고 함
- 연관분석 : 동시에 구매 될 가능성이 큰 상품을 찾아 내어 매출의 증대에 기여하는 분석
- 서점 : 특정 책을 구입한 고객이 어떤 책을 구입했는지, 어떤 책에 관심이 있는지 연결고리를 만듦
- 신용카드 : 수 많은 매출 데이터를 표본으로 도난 카드의 사용 유형 패턴 발견

###  연관규칙의 예
-  금요일, 일반 소매점 고객들이 일회용 기저귀와 캔맥주를 함께 구입
![](07_01.jpg)


###  연관규칙 해석
- 일주일간 쓸 물건을 금요일에 구입하는데 젊은 부부들은 아이를 위해 기저귀를 구매하고, TV 앞에서 마실 맥주도 구입한다는 것이 연관규칙에 의해 얻어짐
- 이러한 정보를 통해 상품을 진열 할 때 기저귀 옆에 맥주, 유아용품 또는 감자칩 등을 진열하여 매출을 증가 시킬 수 있는 매장의 레이아웃이 됨

### 연관규칙 평가
 - 지지율( Support ) 
 - 신뢰도( Confidence ) 
 - 향상도( Lift, Improvement )

### 지지율( Support ) 

![](07_02.jpg)

![](07_03.jpg)

####  지지율의 단점
- 표본수가 적은 경우
    - 연관관계에 대한 통계적 유의성을 증명하기 어려움
    - 투자한 시간, 비용에 비해 판매량을 증가시키는 기여도가 작음
    
#### 지지율의 목적
- 자주 발생하지 않는 규칙을 제거하는데 사용

### 신뢰도( Confidence ) 

![](07_04.jpg)

### 향상도( Lift, Improvement )

![](07_05.jpg)

## Market Basket Analysis Using MapReduce

- 품목이 많고, 거래가 많은 상점의 연관분석할때는 계산이 기하급수적으로 늘어나서, 병렬처리 방식이 필요함.

![](07_06.jpg)

- 위와 같이 A, B, C... 상품의 빈도, ( A, B) , ( A, C)상품의 빈도를 구하면, 이를 이용해서 지지율, 신뢰도, 향상도를 쉽게 구할 수 있음.

## 위의 알고리즘을 pySpark로 구현 가능하지만,  Spark ML에 이미 구현된 것을 사용하는 방법을 알아보자.

- Sparklyr 패키지를 활용 : http://spark.rstudio.com/ 
- SparkR 패키지와 기능을 비슷하지만, Spark ML 및 Spark의 여러 함수들을 호출할 수 있는 Extention을 제공함.

```
# Sparklyr을 우분투에 설치에 필요한 패키지들  
#apt-get -y build-dep libcurl4-gnutls-dev
#apt-get -y install libcurl4-gnutls-dev

# sparklyr은 github으로 배포되기 때문에 devtools을 먼저 설치
install.packages("devtools")
devtools::install_github("rstudio/sparklyr")


library(sparklyr)
spark_install(version = "1.6.1")   # spark을 RStudio에서 설치함

# 자바가 미리 설치되어 있어야 함.
# master = "local" 은 로컬머신에 spark을 구동함.
# master = "yarn or mesos"로 하면 클러스터로 구성된 spark에 연결해서 구동하는 방식
sc <- spark_connect(master = "local")


install.packages("nycflights13")
install.packages("Lahman")

library(dplyr)
iris_tbl <- copy_to(sc, iris)
flights_tbl <- copy_to(sc, nycflights13::flights, "flights")
batting_tbl <- copy_to(sc, Lahman::Batting, "batting")

src_tbls(sc)

# Using dplyr
flights_tbl %>% filter(dep_delay == 2)


delay <- flights_tbl %>% 
  group_by(tailnum) %>%
  summarise(count = n(), dist = mean(distance), delay = mean(arr_delay)) %>%
  filter(count > 20, dist < 2000, !is.na(delay)) %>%
  collect

install.packages("ggplot2")
library(ggplot2)
ggplot(delay, aes(dist, delay)) +
  geom_point(aes(size = count), alpha = 1/2) +
  geom_smooth() +
  scale_size_area(max_size = 2)


# Using SQL

library(DBI)
iris_preview <- dbGetQuery(sc, "SELECT * FROM iris LIMIT 10")
iris_preview


#
# Machine Learning
#

mtcars_tbl <- copy_to(sc, mtcars)
partitions <- mtcars_tbl %>%
  filter(hp >= 100) %>%
  mutate(cyl8 = cyl == 8) %>%
  sdf_partition(training = 0.5, test = 0.5, seed = 1099)

fit <- partitions$training %>%
  ml_linear_regression(response = "mpg", features = c("wt", "cyl"))
fit

summary( fit )


#
# Extensions
# 
library(sparklyr)

# write a csv
tempfile <- tempfile(fileext = ".csv")
write.csv(nycflights13::flights, tempfile, row.names = FALSE, na = "")

# define an R interface to Spark line counting
count_lines <- function(sc, path) {
  spark_context(sc) %>% 
    invoke("textFile", path, 1L) %>% 
    invoke("count")
}

# call spark to count the lines in the csv
count_lines(sc, tempfile)

```


- Sparklyr의 소스코드 
    - https://github.com/rstudio/sparklyr
- ml_linear_regression()함수가 어떻게 만들어졌는지 알 수 있음.
   - https://github.com/rstudio/sparklyr/blob/master/R/ml_linear_regression.R